In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

%matplotlib inline

<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# Recommendation Engines

_Author: Alex Combs (NYC) _

---

<a id="learning-objectives"></a>
### Learning Objectives
*After this lesson, you will be able to:*
- Explain what a recommendation engines is
- Explain the math behind recommendation engines
- Explain the types of recommendation engines and their pros and cons

<a id="what-is-a-recommendation-engine"></a>
## What is a recommendation engine?
---

At its most basic: A system designed to match users to things that they will like.

- The "things" can be products, brands, media, or even other people. 
- Ideally, they should be things the user doesn't know about. 
- **The goal is to rank all the possible things that are available to the user and to only present the top items**

<a id="why-bother"></a>
### Why bother?

- 1/4 to a 1/3 of consumer choices at Amazon are driven by personalized recommendations
- Netflix says there recommendation engine reduces churn saving the company in excess of $1 billion a year
- Hulu [has shown](http://tech.hulu.com/blog/2011/09/19/recommendation-system.html) that showing recommended TV shows results in over 3x more clicks than only showing the most popular TV shows.

<a id="who-uses-recommendation-systems"></a>
### Who uses recommendation systems?

- Netflix
- Pandora
- Hulu
- Tinder
- Facebook
- Barnes & Noble (receipts recommend other books)
- Target (sent directed ads based on motherhood predictions)

<a id="explicit-data-vs-implicit-data"></a>
### Explicit data vs Implicit data

#### Explicit
- Explicity given/pro-actively acquired
- Clear signals
- Cost associated with acquisition (time/cognitive)
- Limited and sparse data because of this


#### Implicit
- Provided/collected passively (digital exhaust)
- Signals can be difficult to interpret
- Enormous quantities

<a id="two-classical-recommendation-methods"></a>
### Two classical recommendation methods

- **Collaborative Filtering**: _(similar people)_
    - If you like the same 5 movies as someone else, you'll likely enjoy other movies they like.
    - There are two main types: (a) Find users who are similar and recommend what they like (**user-based**), or (b) recommend items that are similar to already-liked items (**item-based**).
   

- **Content-Based Filtering** _(similar items)_
    - If you enjoy certain characteristics of movies (e.g. certain actors, genre, etc.), you'll enjoy other movies with those characteristics.
    - Note this can easily be done using machine learning methods! Each movie can be decomposed into features. Then, for each user we compute a model -- the target can be a binary classifier (e.g. "LIKE"/"DISLIKE") or regression (e.g. star rating).

<a id="user-based-collaborative-filtering"></a>
## User-based Collaborative Filtering
---

We'll first look at user-based filtering. The idea behind this method is finding your taste **doppelgänger**. This is the person who is most similar to you based upon the ratings both of you have given to a mix of products.

<a id="so-lets-see-how-we-construct-it"></a>
## So, let's see how we construct it

We begin with what's called a utility matrix. This is a **user** (rows) x **product** (columns) matrix.

In [ ]:
movies = ["user", "Friday the 13th", "Nightmare on Elm St", "Dawn of the Dead", "Hiro Dreams of Sushi", "180 South", "Exit Through the Giftshop"]
users = [
    ("Chuck", 5, 4, None, None, None, 1),
    ("Nancy", 5, None, 4, None, 2, None),
    ("Anya", 4, 5, 5, None, 1, None),
    ("Divya", 1, None, 2, 5, 4, 5),
    ("Pat", 1, 1, 1, None, 3, 4),
]

users = pd.DataFrame(users, columns=movies)
users = users.set_index("user")
users

***Check:*** If we want to find the most similar users, what do we need?

<a id="formula"></a>
If we want to find the users most similar to user A, we need a **similarity metric**.

One metric we can use is **cosine similarity**. Cosine similarity uses the cosine between two vectors to compute a scalar value that represents how closely related these vectors are. 

## $$
cos(\theta) = \frac{\vec{Chuck} \cdot \vec{Nancy}}{\left\| \vec{Chuck}\right\| \left\| \vec{Nancy}\right\| } \
= \frac{\sum{Chuck_i Nancy_i}}{\sqrt{\sum{Chuck_i^2}}\sqrt{\sum{Nancy_i^2}}}
$$

- Angle of $0^{\circ}$ (same direction): $\cos(0^{\circ}) = 1$. Perfectly similar.
- Angle of $90^{\circ}$ (orthogonal): $\cos(90^{\circ}) = 0$. Totally dissimilar.
- Angle of $180^{\circ}$ (opposite direction): $\cos(90^{\circ}) = -1$. Opposite.


Doesn't this sound a lot like the correlation coefficient? It turns out that cosine similarity is identical to the **uncentered correlation coefficient**! As a bonus, if the points are mean-centered, then this formula also depicts the **Pearson correlation coefficient**.

<a id="cosine-similarity-using-sci-kit-learn"></a>
## Cosine similarity using sci-kit learn

With that, let's calculate the cosine similarity of A against all other users. We'll start with B. We have a sparse matrix with lot's of missing values... what should we do?

In [ ]:
users

In [ ]:
users.mean(axis=0)

In [ ]:
users.mean(axis=1)

In [ ]:
def mean_center_rows(df):
    return (df.T - df.mean(axis=1)).T

users_mc = mean_center_rows(users)
users_mc

In [ ]:
users_mc = users_mc.fillna(0)

Now let's calculate the (cosine) similarity for...

1. Chuck vs. Nancy
2. Chuck vs. Divya

Do these results make sense??

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
Chuck_mc = users_mc.loc['Chuck',:].values.reshape(1,-1)
Nancy_mc = users_mc.loc['Nancy',:].values.reshape(1,-1)
Divya_mc = users_mc.loc['Divya',:].values.reshape(1,-1)

print('Chuck vs. Nancy = ', cosine_similarity(Chuck_mc, Nancy_mc))
print('Chuck vs. Divya = ', cosine_similarity(Chuck_mc, Divya_mc))

In [ ]:
sim_matrix = cosine_similarity(users_mc)
users_sim = pd.DataFrame(sim_matrix, columns=users_mc.index, index=users_mc.index)
users_sim

In [ ]:
sns.heatmap(users_sim, annot=True, cmap='coolwarm')

<a id="but-how-do-we-predict-the-rating-of-an-item-for-a-user"></a>
## But how do we predict the rating of an item for a user?

In [ ]:
users

**Will Nancy like Nightmare on Elm St?**

To answer this we will predict Nancy's rating using the _weighted results of the other users_.

This is where a little art comes in... let's make the decision to only include users that have a positive similarity score relative to Nancy.

In [ ]:
users_sim

The gist of what we want to do is given by the following formulaL

$$\frac{(\text{Chuck-Nancy sim})(\text{Chuck rating}) + (\text{Nancy-Anya sim})(\text{Anya ratings})}{\text{sum of similarities}}$$

$$= \frac{0.349428 \cdot (4) + 0.129503 \cdot (5)}{0.349428 + 0.129503} = 4.68$$

Here's the procedure for recommending movie X to Nancy:

1. Get Nancy's similarity scores to all other **users**.
2. Only keep positive similarities.  (This step can be tweaked!)
3. Convert Nancy's similarities to _weights_ by dividing each similarity by the _total_ similarity.
4. Get all the other users' ratings for movie X.
5. Weight each user's rating by Nancy's similarity to that user.

In [ ]:
# get Nancy's similarity scores
nancy_sim = users_sim['Nancy'].drop('Nancy')
nancy_sim = nancy_sim[nancy_sim > 0]
nancy_sim

In [ ]:
# Turn Nancy's similarity scores to weights
nancy_weights = nancy_sim.values/np.sum(nancy_sim.values)
nancy_weights

In [ ]:
# Get elm_st_ratings
elm_st_ratings = users['Nightmare on Elm St'].drop('Nancy').loc[nancy_sim.index]
elm_st_ratings

In [ ]:
np.dot(elm_st_ratings.values, nancy_weights)

How would we get expected ratings for all of the movies that Nancy has not seen?

In [ ]:
ratings = users.T
ratings = ratings[ratings['Nancy'].isnull()]
ratings = ratings.drop('Nancy', axis=1)
ratings = ratings[nancy_sim.index]
ratings

In [ ]:
ratings = users.T
ratings = ratings[ratings['Nancy'].isnull()]
ratings = ratings.drop('Nancy', axis=1)
ratings = ratings[nancy_sim.index]

nancy_ratings = np.dot(ratings.fillna(0).values, nancy_weights)
nancy_ratings

In [ ]:
pd.DataFrame(nancy_ratings, index=ratings.index)

#### Check: What might be some problems with user-based filtering?

- Frequently-liked items will necessarily have users who like all kinds of other items. So, recommendations based on frequently-liked items may be inaccurate.

- User-based filtering also suffers from the **cold-start problem**. If a new user joins and has very few likes, then it is difficult to pair them with a similar user.

- Lastly, suppose that a user with few likes adds a new like. This may significantly change the recommendations. Hence, as users add likes, the recommendations must be continually and quickly updated.

In practice, there is a type of collaborative filtering that can perform much better than user-based filtering: **item-based filtering**.

<a id="item-based-collaborative-filtering"></a>
## Item-based Collaborative Filtering

In item-based filtering, we are trying to find similarities across items rather than users.

> Rather than looking for _similar users_, we are going to look for _similar movies/items_.  Similar items are determined by user rating patterns.

Basically, everything here is the same as in user-based collaborative filtering, except we will deal with the _transpose_ or our data matrix.

Just as in user-based filtering, we need to center our values by row.

In [ ]:
sim_matrix = cosine_similarity(mean_center_rows(users.T).fillna(0))
movies_sim = pd.DataFrame(sim_matrix, columns=users.T.index, index=users.T.index)
movies_sim

The nearest movies to _Nightmare on Elm St_ are _Firday the 13th_ and _Dawn of the Dead_. To calculate Nancy's rating for our _Nightmare on Elm St_, we have the following equation:

** (1st closest cosine sim movie x Nancy's rating + 2nd closest cosine sim movie x Nancy's product rating) / (sum of 1st and 2nd's cosine sims) **

$$\frac{(\text{Dawn of Dead sim})(\text{Nancy's rating}) + (\text{Friday 13 sim})(\text{Nancy's rating})}{\text{sum of similarities}} = \frac{0.859338 \cdot (4) + 0.635369 \cdot (5)}{0.859338 + 0.635369} = 4.43$$

Here's the procedure for recommending movie X to Nancy:

1. Get movie X's similarity scores to all other **movies**.
2. Only keep positive similarities.  (This step can be tweaked!)
3. Convert movie X's similarities to _weights_ by dividing each similarity by the _total_ similarity.
4. Get Nancy's ratings for all other movies.
5. Weight Nancy's ratings for all other movies by movie X's similarity to those movies.

In [ ]:
# get Elm St similarity scores
elm_st_sim = movies_sim['Nightmare on Elm St'].drop('Nightmare on Elm St')
elm_st_sim = elm_st_sim[elm_st_sim > 0]
elm_st_sim

In [ ]:
# Turn Elm St similarity scores to weights
elm_st_weights = elm_st_sim.values/np.sum(elm_st_sim.values)
elm_st_weights

In [ ]:
# Get Nancy's_ratings
nancy_ratings = users.T['Nancy'].loc[elm_st_sim.index]
nancy_ratings

In [ ]:
np.dot(nancy_ratings.values, elm_st_weights)

<a id="content-based-filtering"></a>
## Content-based Filtering

Finally, there is another method called content-based filtering. In content-based filtering, the items are broken down into "feature baskets". These are the characteristics that represent the item. The idea is that if you like the features of song X, then finding a song that has similar characteristics will tell us that you're likely to like it as well.


The quintessential example of this is Pandora with it's musical genome. Each song is rated on ~450 characteristics by a trained musicologist.

<a id="example"></a>
## Example 
Content-based filtering begins by mapping each item into
a feature space. Both users and items are represented by
vectors in this space.
Item vectors measure the degree to which the item is
described by each feature, and user vectors measure a
user’s preferences for each feature.
Ratings are generated by taking dot products of user &
item vectors. 

<img src="http://i.imgur.com/NzHksKK.png">

---

Let's look at a music database as an example

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd, numpy as np
import sqlite3
conn = sqlite3.connect("./db.sqlite3")
conn.text_factory = lambda x: str(x, 'latin1')

sql = """
SELECT r.userID, r.artistID, r.tagID, 
a.name AS artist,
t.tagValue as genre
FROM rec_user_artist_tags r
LEFT JOIN rec_artists a on r.artistID = a.id
LEFT JOIN rec_tags t on r.tagID = t.tagID
WHERE a.name NOT NULL
LIMIT 15000
"""

artists = pd.read_sql(sql, con=conn)
artist_genre = artists.groupby(["artist", "genre"]).size().sort_values(ascending=False).unstack().fillna(0)

In [ ]:
artists.info()

In [ ]:
artists.shape

In [ ]:
artist_genre.head()

### Basic Similarity Metric: "Give me something like Linkin Park!"

In [ ]:
artist_sim = pd.DataFrame(
    cosine_similarity(artist_genre),
    columns=artist_genre.index,
    index=artist_genre.index
)

In [ ]:
artist_sim['ABC'].sort_values(ascending=False).head(10)
#artist_sim['Aidan Baker'].sort_values(ascending=False).head(10)
# artist_sim['Linkin Park'].sort_values(ascending=False).head(10)
# artist_sim['2Pac'].sort_values(ascending=False).head(10)
# artist_sim['Justin Bieber'].sort_values(ascending=False).head(10)

#artist_sim['Coldplay'].sort_values(ascending=False).head(10)
#artist_sim['Lady Gaga'].sort_values(ascending=False).head(10)
# artist_sim['Michael Jackson'].sort_values(ascending=False).head(5)

### Profile-Based Recommendations

In [ ]:
genres = artist_genre.columns.tolist()
my_profile = pd.Series(np.zeros(len(genres)), index=artist_genre.columns)

my_profile['rock'] = 4
my_profile['world'] = 3
my_profile['folk'] = 3
my_profile['electronic'] = 1

print(len(my_profile))

recommendations = np.dot(artist_genre.values, my_profile.values)
recommendations = pd.Series(recommendations, index=artist_genre.index)
recommendations.sort_values(ascending=False).head(20)

<a id="conclusion"></a>
## Conclusion
---

We have looked at the major types of recommender systems in this lesson. Let's quickly wrap up by looking at the pros and cons of each.

#### Collaborative Filtering 

Pros:
- No need to hand craft features

Cons:
- Needs a large existing set of ratings (cold-start problem)
- Sparsity occurs when the number of items far exceeds what a person could purchase

#### Content-based Filtering

Pros:
- No need for a large number of users

Cons:
- Lacks serendipity
- May be difficult to generate the right features
- Hard to create cross-content recommendations (different feature spaces)

In fact, the best solution -- and the one most likely in use in any large-scale, production system is a combination of both of these. This is known as a **hybrid system**. By combining the two systems, you can get the best of both worlds.

<a id="extra-practice"></a>
## Extra Practice
---

Using the [MovieLens dataset](https://grouplens.org/datasets/movielens/100k/), experiment with building a recommender system. Check the "Additional Resources" for more information and some considerations on how to evaluate these systems.

<a id="additional-resources"></a>
## Additional Resources
---

- [Wharton Study of Recommender Systems](http://knowledge.wharton.upenn.edu/article/recommended-for-you-how-well-does-personalized-marketing-work/)
- [Netflix Recommendations](https://www.rtinsights.com/netflix-recommendations-machine-learning-algorithms/)
- [Netflix Paper](http://dl.acm.org/citation.cfm?id=2843948)
- [NY Times Rec System](https://open.blogs.nytimes.com/2015/08/11/building-the-next-new-york-times-recommendation-engine)
- [Evaluating Rec Systems](https://www.quora.com/How-do-you-measure-and-evaluate-the-quality-of-recommendation-engines)